## NLP Tools for Digital Humanities Final Project

# Explicit Content in the 2020 Wattys Award Winners Books

### Getting the data
<p>I used my own fork of Wattpad2Epub script. Wattpad2Epub allows you download Wattpad stories easily and save them in epub format.</p>
<p>Then I needed to convert epub to txt, so I can process it with Python. I used Calibre for this task.</p>

### Cleaning the data
<p>Every book starts with Introduction containing synopsis, chapter list, and info about the book. I wanted as clean data as I could so I decided to delete these. Unfortunately, every book contains a bit different Introduction, so I had to go over them manually.</p>
<p>I was not sure if I wanted to leave authors' notes in the books, then I decided to remove them, too. I had to remove them manually, too.</p>
<p>Oops! This image does not follow our content guidelines. To continue publishing, please remove it or upload a different image.</p>

<p>Read all files</p>